In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import torch

# 基础模型路径
base_model_path = "/kaggle/input/qwen2.5/transformers/7b-instruct/1"
# LoRA 适配器路径
lora_adapter_path = "/kaggle/input/opendeepdiver-sft-lora/qwen2.5-7b-lora-adapter"
# 合并后模型的保存路径
save_path = "/kaggle/working/merged_model"

# 加载基础模型和分词器
print("加载基础模型...")
model = AutoModelForCausalLM.from_pretrained(
    base_model_path,
    torch_dtype=torch.float16,  # 根据需求调整 dtype（如 float32）
    device_map="auto",
    trust_remote_code=True  # 如果 Qwen 需要此参数
)
tokenizer = AutoTokenizer.from_pretrained(base_model_path, trust_remote_code=True)

# 加载 LoRA 适配器
print("加载 LoRA 适配器...")
model = PeftModel.from_pretrained(model, lora_adapter_path)

# 合并模型（将 LoRA 权重合并到基础模型中）
print("合并 LoRA 适配器...")
model = model.merge_and_unload()

# 保存合并后的模型
print(f"保存合并后的模型到 {save_path}...")
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print("合并完成！")